## Genarate masks for ROIs

This notebooks takes the curated point corresponding to the centroids of the nuvlei and generates a mask.
The mask if fomed by running a region growing algorithm for on small region centered around each centroid.

This notebook also uses Napari inside Julia, which requires bulding `PyCall` against the python in current poetry environment

In [1]:
]activate ../

Activating environment at `~/projects/borton/calcium-roi-analysis/Project.toml`


In [2]:
using NPZ
using Revise
using YAML
Revise.includet("../julia_src/mask.jl")
Revise.includet("../julia_src/utils.jl")

In [3]:
using PyCall

# Needed for napari
pygui_start(:qt5)
# needed to kickstart Qt5 for some reason
qapp_obj = pyimport("qtpy.QtWidgets").QApplication([""])
napari = pyimport("napari")
skimage = pyimport("skimage")

PyObject <module 'skimage' from '/Users/mrestrep/projects/borton/calcium-roi-analysis/.venv/lib/python3.8/site-packages/skimage/__init__.py'>

## Read parameters from file

In [4]:
compute_mask_data = false
mask_radius = 10

10

In [6]:
params, data_path, output_path = prepare_paths("../parameters/current.yaml")

(Dict{Any,Any}("radius" => [5.0, 7.0, 9.0],"output_base_path" => "/Users/mrestrep/projects/borton/calacium_images/sophie_data/julia_output","input_tiff_dir" => "/Users/mrestrep/projects/borton/calacium_images/sophie_data/tiffs"), "/Users/mrestrep/projects/borton/calacium_images/sophie_data/tiffs", "/Users/mrestrep/projects/borton/calacium_images/sophie_data/julia_output")

# Read the Volume of Interest (VOI)

In [7]:
voi = load_tiffstack(data_path);
voi_napari = convert(Array{Float64}, voi);
radius = params["radius"]

3-element Array{Float64,1}:
 5.0
 7.0
 9.0

# Load curated data

In [8]:
curated_file = output_path * "/curated_blobs.npz"
curated_data = npzread(curated_file)
seed_coords = curated_data["coords"][curated_data["class_label"], :]

110×3 Array{Int64,2}:
 225  325  11
  41  313  12
 121  345  12
  68  220  13
  74  468  13
  21  267  14
 353  365  14
 255  364  16
 403  426  16
 284  356  17
 291  401  17
 323  457  17
 155  190  18
   ⋮         
  62   42  44
  16   83  44
  11  106  45
  56   21  46
 238   49  46
 244  112  46
 426  294  46
 247   89  48
 512  255  48
 115   11  54
 205   97  54
 231  118  54

In [9]:
masked_voi = Array{Float64,3}[]
voi_labels = Array{Int64,3}[]

mask_file = output_path *  "/masked_voi.npy"
labels_file = output_path  * "/labels_voi.npy"


if compute_mask_data
    @time voi_labels, masked_voi = subregion_collage(voi_napari, float.(seed_coords), mask_radius)
    @time npzwrite(mask_file, masked_voi)
    @time npzwrite(labels_file, voi_labels)
else
    masked_voi = npzread(mask_file)
    voi_labels = npzread(labels_file)
end


512×512×54 Array{Int64,3}:
[:, :, 1] =
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  

# Visualize Data

In [10]:
viewer = napari.view_image(voi_napari, rgb=false, name="raw")
viewer.add_image(masked_voi, name = "mask")
viewer.add_labels(voi_labels, name = "segments")
viewer.add_points(seed_coords, size=3, face_color = "#2aff05", edge_color = "#2aff05", name = "centroids")

PyObject <Points layer 'centroids' at 0x190eabd00>